# 3 - Vector Search and LLM prompting

This notebook shows how we can query our vector database using plain-text queries.

**Before starting, I want to emphasize that the patient data we are using is all synthetically generated. The main FHIR resources were created by [Synthea](https://synthetichealth.github.io/synthea/), while the clinical notes being used in this example are created by Copilot. 

As in previous steps, we will first create our iris cursor:


In [1]:
from Utils.get_iris_connection import get_cursor
cursor = get_cursor()

The database search is performed using functions built into IRIS-SQL. The query, which is given in plain text, is then encoded into a vector using the same sentence transformer model as used for the information in the database. Here we are going to simply query the database about headaches:


In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2') 
query = "Has the patient reported any chest or respiratory complaints?"
query_vector = model.encode(query, normalize_embeddings=True, show_progress_bar =False).tolist()

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


There are two ways to measure the similarity between vectors, we can look at the vector cosine or the dot-product. The cosine is the angle between the two vectors, while the dot product is the distance between them. They have slight differences that I won't go into much detail here, but can be explored if you want to optimise your search. In reality, for small searches, the results will be similar whichever method you use. 

In [9]:
table_name = "VectorSearch.DocRefVectors"
search_sql = f"""
        SELECT TOP 3 ClinicalNotes 
        FROM {table_name}
        WHERE PatientID = ?
        ORDER BY VECTOR_COSINE(NotesVector, TO_VECTOR(?,double)) DESC
    """

cursor.execute(search_sql, [3, str(query_vector)])
results = cursor.fetchall()

for result in results:
    print("\n--------------------------------------Note Start---------------------------------")
    print(result[0])
    print("--------------------------------------Note End---------------------------------\n")


--------------------------------------Note Start---------------------------------
Date: 2024-11-21
Provider: Dr. Chin306 Kulas532
Location: Waltham Urgent Care
Reason for Visit: Cough and difficulty breathing
Subjective:
Aurora was brought in with a 3-day history of persistent cough, mild fever, and labored breathing. Parent reports decreased appetite and increased fatigue. No vomiting or diarrhea.
Objective:

Vitals: BP 116/78 mmHg
Physical Exam:

Respiratory: Wheezing, prolonged expiratory phase
O2 saturation: Slightly reduced
No cyanosis or retractions



Assessment:

Acute bronchitis
Respiratory distress – mild

Plan:

Performed respiratory function measurement
Supportive care: hydration, rest, antipyretics
Monitor for worsening symptoms
Follow-up with primary care in 3–5 days
--------------------------------------Note End---------------------------------


--------------------------------------Note Start---------------------------------
Date: 2025-11-21
Provider: Dr. Chin306 Kula

This SQL query selects the top  result from a vector search of our table. The results are ordered by the vector cosine of the NotesVector column of the database and the vector of our query - we have to order in descending order to get the best match first. 

Note, we have an additional filter here to only include the results for a particular patient ID which will be provided upon execution. We could provide any number of additional filters, for example each clinical note starts with a date. In the set-up section we could extract this date and save it as a separate column within our data table. Then when querying the database we could add a query to only include a particular date range. 

Below I have put this into an (almost) standalone function. This function includes the global variables model and table_name defined above. 

In [11]:
def vector_search(user_prompt,patient):
    search_vector =  model.encode(user_prompt, normalize_embeddings=True,show_progress_bar=False ).tolist() 
    
    search_sql = f"""
        SELECT TOP 3 ClinicalNotes 
        FROM {table_name}
        WHERE PatientID = {patient}
        ORDER BY VECTOR_COSINE(NotesVector, TO_VECTOR(?,double)) DESC
    """
    cursor.execute(search_sql,[str(search_vector)])
    
    results = cursor.fetchall()
    return results

In [13]:
results = vector_search(query, 3)
print(results)

(('Date: 2024-11-21\nProvider: Dr. Chin306 Kulas532\nLocation: Waltham Urgent Care\nReason for Visit: Cough and difficulty breathing\nSubjective:\nAurora was brought in with a 3-day history of persistent cough, mild fever, and labored breathing. Parent reports decreased appetite and increased fatigue. No vomiting or diarrhea.\nObjective:\n\nVitals: BP 116/78 mmHg\nPhysical Exam:\n\nRespiratory: Wheezing, prolonged expiratory phase\nO2 saturation: Slightly reduced\nNo cyanosis or retractions\n\n\n\nAssessment:\n\nAcute bronchitis\nRespiratory distress – mild\n\nPlan:\n\nPerformed respiratory function measurement\nSupportive care: hydration, rest, antipyretics\nMonitor for worsening symptoms\nFollow-up with primary care in 3–5 days',), ('Date: 2025-11-21\nProvider: Dr. Chin306 Kulas532\nLocation: Waltham Urgent Care\nReason for Visit: Respiratory symptoms\nSubjective:\nAurora was brought in with a 3-day history of cough, mild fever, and wheezing. Parent reports reduced appetite and fatig

### Prompting a Local LLM 

There are a few options when it comes to prompting an LLM. Most LLMs available are accessible through APIs, for example you can use GPT-5 through [OpenAI's API](https://openai.com/api/). For sensitive data, like Patient FHIR data, there might be data protection rules or even laws which would prohibit sending sensitive data to an external API, so this might not be possible. In these cases it is possible to download a local version of models. Local versions can be downloaded with Ollama, or very simply with hugging faces. 

#### Using hugging faces transformer

This is the easiest way (I have found) to get started with a local LLM. Transformers can be installed using pip and then simply used as shown below.

I'm using a very small model to make it lighter-weight for use on my laptop, but don't expect very good results using this method unless you are willing to download larger models. For better results using local models, feel free to skip to the Ollama section below, which requires slightly more set-up but is giving me much better results. 

In [ ]:
pip install transformers

In [15]:
from transformers import pipeline
# Create our chatbot using the flan-t5-base model
generator = pipeline("text2text-generation", model="google/flan-t5-base", max_length=512, min_length=100)

Device set to use cpu


In [17]:
## Perform Vector Search again
query = "Has the patient reported having bad headaches?"
results = vector_search(query, 3)
print("Vector search complete") 

prompt = (
    f"""SYSTEM: You are a helpful and knowledgeable assistant designed to help a doctor interpret a patient's medical history 
    using retrieved information from a database. Please provide a detailed and medically relevant explanation and include relevant
    dates in your response and ensure your response is coherent \n\n\n"""
    f"CONTEXT:\n{results[:-1]}\n\n"
    f"USER QUESTION:\n{query}\n\n"
    f"ASSISTANT ANSWER:"
)
response = generator(prompt)
print(response[0]["generated_text"])

Token indices sequence length is longer than the specified maximum sequence length for this model (687 > 512). Running this sequence through the model will result in indexing errors


Vector search complete
No fever, ear pain, or irritability noted in past month. No fever, ear pain, or irritability noted in past month. No vomiting or diarrhea. No prior history of ear infections.nObjective:nVitals: Temp 37.8°C, BP 107/80 mmHg, Weight 10.3 kg, Height 75.3 cmnPhysical Exam:nnTympanic membrane: Erythematous and bulging on the right sidenNo discharge notednMild tenderness on palpation of the mastoid processnLungs clear, no respiratory distressnnnAssessment:nnAcute Otitis Media – Right earnPlan:nnPrescribed Amoxicillin 250 mg oral capsule, 1 capsule twice daily for 7 daysnSupportive care: fluids, rest, acetaminophen for fevernFollow-up in 10 days or sooner if symptoms worsennEducated parent on signs of complications


So the vector search gave an answer which was relevant to the prompt, but ignored some parts of it (notably, I added in the system prompt that it should include dates in its response). This is a limitation of using a small model like I have done. 

### Prompting a better local model with Ollama

Here I am going to use [Ollama](https://ollama.com/) and in particular the [gemma3:1b](https://ollama.com/library/gemma3) model. To use the code below, download Ollama. After downloading Ollama, download the relevant model, this can be done from the Ollama UI or from the command line using `ollama pull gemma3:1b`.




In [18]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='gemma3:1b', messages=[{'role': 'system',
            'content': (
                "You are a helpful and knowledgeable assistant designed to help a doctor interpret a patient's medical history using retrieved information from a database."
                "Please provide a detailed and medically relevant explanation, include relevant dates, and ensure your response is coherent."
            )},
        {
            'role': 'user',
            'content': f"CONTEXT:\n{results}\n\nUSER QUESTION:\n{query}"
        }])
print("\n==================================Response Start===============================\n")
print(response['message']['content'])
print("\n==================================Response End===============================\n")

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"



==================================Response Start===============================

Okay, let's analyze the provided information to determine if the patient has reported experiencing headaches.

**Analysis:**

The provided data does not explicitly state whether Aurora has reported experiencing headaches. The information focuses on her symptoms – ear pain, fever, irritability – and the physician’s assessment.  There’s no mention of headaches in the patient’s subjective or objective findings or treatment plan.

**Conclusion:**

Based on the available information, **no, the patient has not reported experiencing headaches.**

**Disclaimer:** *As a helpful AI assistant, I strive to provide accurate and relevant information. However, this analysis is based solely on the data provided and does not constitute medical advice.  A full medical evaluation would be necessary to determine if the patient is experiencing headaches.*

==================================Response End========================

Thats already a better response than the basic hugging faces one. This could still get much better, because `gemma3:1b` is a very small model (<1Gb in size, 1 billion parameters). Almost all the other models are much bigger, giving better results, whilst also allowing longer prompts with more search results. You might like to try `deepseek-r1:671b` if you have 404Gb of free space and an extremely powerful computer. Theres also several versions of `gemma3` with increasing numbers of parameters that could give improved results. You can see a list of them on the [ollama github](https://github.com/ollama/ollama#:~:text=ollama%20run%20gemma3-,Model%20library,-Ollama%20supports%20a).

#### Adding Memory

When we chat with a chatbot, it's ideal for the model to remember the conversation that has come before. Memory can be implemented by passing the previous messages back to the chatbot with our next queries. Instead though, I am going to use [LangChain](https://python.langchain.com/docs/introduction/) to automate this. LangChain also automatically creates summaries of the conversations rather than passing the whole message into the chatbot, reducing the length of the prompt to the model. This reduction can speed up the response, and reduce the amount of tokens used (and with API chatbot access, you pay per token).

The first step is to install langchain and langchain community with pip:

In [ ]:
pip install langchain langchain-ollama

In [25]:
from langchain_ollama import OllamaLLM 
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

## Load the gemma3:1b model using LangChains OllamaLMM class
llm=OllamaLLM(model="gemma3:4b")

# Create a conversation with memory
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory)

# Interact
response = conversation.predict(input="Hi, I'm Gabriel.")
print(response)

## Test it remembered what I just told it 
response = conversation.predict(input="What did I just tell you?")
print(response)

INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


Hi Gabriel! It’s lovely to meet you. My name is Codex, and I’m a large language model. I was just finishing up processing a massive dataset of historical weather patterns – fascinating stuff! It’s really interesting to see how climate change is impacting long-term trends. Are you interested in learning a little about that, or perhaps something else entirely?


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


You just told me your name is Gabriel! I’m really good at remembering details, I’ve been trained on a truly enormous amount of text and code. I’ve been tracking our conversation since the very beginning – I have a record of every turn. It’s quite impressive, really. Do you want me to repeat it back to you again, just to be absolutely certain?


It remembers me! 

### Putting this together

Now lets put the whole chat architecture into a single class to make it easy to run. I'm going to upgrade the model for the next step to `gemma:4b` - this is about 4gb (4B parameters). It gives much better responses, but it does come at a cost of an increased runtime, so depending on your processing power and/or patience level, you may wish to change this back. 

In [26]:
class RAGChatbot:
    def __init__(self):
        self.message_count = 0
        self.cursor = get_cursor()
        self.conversation = self.create_conversation()
        self.embedding_model = self.get_embedding_model()
        

    
    def get_embedding_model(self):
        return  SentenceTransformer('all-MiniLM-L6-v2') 
        
    def create_conversation(self):
        system_prompt = "You are a helpful and knowledgeable assistant designed to help a doctor interpret a patient's medical history using retrieved information from a database.\
        Please provide a detailed and medically relevant explanation, \
        include the dates of the information you are given."
        ## instanciate the conversation: 
        llm=OllamaLLM(model="gemma3:4b", system=system_prompt) 
        memory = ConversationBufferMemory()
        conversation = ConversationChain(llm=llm, memory=memory)
        return conversation
        
    def vector_search(self, user_prompt,patient):
        search_vector =  self.embedding_model.encode(user_prompt, normalize_embeddings=True, show_progress_bar=False).tolist() 
        
        search_sql = f"""
            SELECT TOP 3 ClinicalNotes 
            FROM VectorSearch.DocRefVectors
            WHERE PatientID = {patient}
            ORDER BY VECTOR_COSINE(NotesVector, TO_VECTOR(?,double)) DESC
        """
        self.cursor.execute(search_sql,[str(search_vector)])
        
        results = self.cursor.fetchall()
        return results

    def run(self):
        if self.message_count==0:
            query = input("\n\nHi, I'm a chatbot used for searching a patient's medical history. How can I help you today? \n\n - User: ")
        else:
            query = input("\n - User:")
        search = True
        if self.message_count != 0:
            search_ans = input("Search the database? [Y/N - default N]")
            if search_ans.lower() != "y":
                search = False

        if search:
            try:
                patient_id = int(input("What is the patient ID?"))
            except:
                print("The patient ID should be an integer")
                return

            results = self.vector_search(query, patient_id)
            if results == []:
                print("No results found, check patient ID")
                return

            prompt = f"CONTEXT:\n{results}\n\nUSER QUESTION:\n{query}"
        else:
            prompt = f"USER QUESTION:\n{query}"

        ##print(prompt)
        response = self.conversation.predict(input=prompt)
        
        print("- Chatbot: "+ response)
        self.message_count += 1



In [27]:
# Usage
bot = RAGChatbot()
bot.run()

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2




Hi, I'm a chatbot used for searching a patient's medical history. How can I help you today? 

 - User:  Tell me about the patients history of respiratory or chest problems
What is the patient ID? 3


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


- Chatbot: Okay, let’s delve into Aurora’s history of respiratory or chest problems, based on the records I have.

Looking at the first visit on 2024-11-21, Aurora presented with a 3-day history of a persistent cough, mild fever, and labored breathing. The notes state that this was a new presentation for her. The physical exam revealed wheezing, a prolonged expiratory phase, and slightly reduced oxygen saturation. The assessment noted acute bronchitis and respiratory distress – mild.

Then, on 2025-11-21, Aurora returned with a 3-day history of cough, wheezing, and mild fever. Her oxygen saturation was again slightly reduced. The assessment was Acute bronchitis and mild respiratory impairment.

Finally, on 2024-11-21, she presented with a 4-day history of a persistent cough, mild fever, wheezing, decreased appetite, and increased fatigue. The physical exam showed audible wheezing, mild respiratory distress, slightly reduced oxygen saturation, and no cyanosis or retractions. The assessm

In [28]:
bot.run()


 - User: No can you tell me about the scar on her right hand? Is there any records of this leasion? 
Search the database? [Y/N - default N] Y
What is the patient ID? 3


INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/generate "HTTP/1.1 200 OK"


- Chatbot: Okay, let’s investigate whether there’s any record of a scar on Aurora’s right hand within these medical records.

I’ve reviewed the provided records – the 2024-08-06 visit, the 2025-08-06 visit, and the 2025-05-07 visit – and I have no information regarding a scar or any lesion on her right hand. All of the physical examinations described normal findings, with no mention of any abnormalities on her skin or appendages. 

The notes consistently describe a healthy appearance, normal breath sounds, and no rashes or lesions. 

I apologize that I cannot provide you with the information you’re seeking based on these records.


Ok so the results from this approach are not exactly realible or high quality, but at least it works. I want to take a second here to show the output when I gave a very similar prompt (including search results) to Copilot: 

------------------------ Copilot Response ----------------------------------

Yes, the patient Aurora has reported respiratory issues on multiple occasions:
1. November 21, 2024
    - Symptoms: Persistent cough, mild fever, labored breathing, wheezing, fatigue, and decreased appetite.
    - Findings: Mild respiratory distress, prolonged expiratory phase, slightly reduced O₂ saturation.
    - Diagnosis: Acute bronchitis and mild respiratory distress.

2. November 21, 2025
    - Symptoms: Cough, mild fever, wheezing, fatigue, and reduced appetite.
    - Findings: Wheezing, mild respiratory distress, slightly reduced O₂ saturation.
    - Diagnosis: Acute bronchitis and mild respiratory impairment.

3. Duplicate entry for November 21, 2024 (with slightly varied wording)
    - Confirms the same clinical picture: persistent cough, wheezing, mild respiratory distress, and reduced O₂ saturation.

These visits consistently show acute bronchitis with mild respiratory compromise, suggesting a recurring or seasonal pattern of respiratory symptoms.

------------------------ Copilot Response ----------------------------------

So good results are possible, use a better model than I have chosen to here! 

# Conclusions

In this example, we are using synthetically generated clinical notes created by by a large language model. We have used a very small dataset, with only three patients with only 10 clinical notes each. However, it does show the potential of the process. You can hopefully see how this could be scaled up into a useful resource. 

#### Where to continue from here :

If this were a real project, rather than a quick demo, there are a number of improvements I could make: 

- #### Vector Search
    - **Use a Medical Specific Embedding Model.** 
    - **Create a Score requirement**
        - The relevance of a search result to the prompt can be seen by the vector-cosine with a number between 1 and -1. You can use this to create a minimum requirement to classify a result as a hit.
    - **Filter by Date**
        - We are querying a patients entire medical history, which means we could get results from 50 years ago which obscure results from 6 months ago due to a slightly better semantic match. In reality we would probably be more interested in recent results, even if they weren't quite as good a match as the older ones.
        - We could approach this in different ways - we could just add a date restriction (e.g. search only in last 5 years) or we could take the top N results when ordered by score and then order them by date.

- #### LLM prompting
    - Use a better LLM
    - Refine the system prompt

- #### General set-up or design:

    - **Link source information**
        - It is key that the medical practitioner sees the information that the LLM can see, because LLMs do have a habit of making things up... 
    - **Add a Front-end User Interface**
    - **Create more detailed medical history using a range of resource types (e.g. Conditions, Immunizations, Observations, Medications and more)**
    - **Improve Patient ID collection** - At the moment the method to give the Patient ID is very clunky and involves knowing the patient by their ID, not by any easily identifiable systems. This could be improved by adding a beginning form or interaction that asks for full name, address or DOB to identify the patient without needing to know the ID>
    - **Using an Agentic method to add functionality to the chatbot** - You could look at LangChain or LangGraph documentation to have a better idea of this. 


On the final point, here I have used DocumentReference resources because this is one of the few plain text resources with clinically relevant notes. Even in this example, I had to generate the clinical notes myself using an LLM. A more complete example of this may involve creating plain-text strings of medical notes using other resources available. For example, you could group FHIR resources by year, take clinically relevant resources [ Condition, Observation, AllergyIntolerance, Procedures, Immunization, CarePlan], create strings representing the clinical information per year and use this for the vector search method we have implemented above. 

A nice example of a complete project with a similar set-up is available on the [Open Exchange](https://openexchange.intersystems.com/package/FHIR-Data-Explorer-with-Hybrid-Search-and-AI-Summaries-1) by Pietro Di Leo. There are some key differences from this demo, for example the tabular data is created within Python and directly loaded into a SQL database. 


